In [1]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn import ensemble
from sklearn import model_selection

from evidently import ColumnMapping
from evidently.options import ColorOptions
from evidently.report import Report

from evidently.metrics import ColumnDriftMetric
from evidently.metrics import DataDriftTable
from evidently.metrics import DatasetDriftMetric
from evidently.metrics import ColumnDistributionMetric
from evidently.metrics import ColumnValuePlot
from evidently.metrics import ColumnQuantileMetric
from evidently.metrics import ColumnCorrelationsMetric
from evidently.metrics import ColumnValueListMetric
from evidently.metrics import ColumnValueRangeMetric
from evidently.metrics import DatasetCorrelationsMetric
from evidently.metrics import ColumnRegExpMetric
from evidently.metrics import ColumnSummaryMetric
from evidently.metrics import ColumnMissingValuesMetric
from evidently.metrics import DatasetSummaryMetric
from evidently.metrics import DatasetMissingValuesMetric
from evidently.metrics import ConflictTargetMetric
from evidently.metrics import ConflictPredictionMetric
from evidently.metrics import ClassificationQualityMetric
from evidently.metrics import ClassificationClassBalance
from evidently.metrics import ClassificationConfusionMatrix
from evidently.metrics import ClassificationQualityByClass
from evidently.metrics import ClassificationClassSeparationPlot
from evidently.metrics import ClassificationProbDistribution
from evidently.metrics import ClassificationRocCurve
from evidently.metrics import ClassificationPRCurve
from evidently.metrics import ClassificationPRTable
from evidently.metrics import ClassificationQualityByFeatureTable
from evidently.metrics import ClassificationDummyMetric
from evidently.metrics import RegressionQualityMetric
from evidently.metrics import RegressionPredictedVsActualScatter
from evidently.metrics import RegressionPredictedVsActualPlot
from evidently.metrics import RegressionErrorPlot
from evidently.metrics import RegressionAbsPercentageErrorPlot
from evidently.metrics import RegressionErrorDistribution
from evidently.metrics import RegressionErrorNormality
from evidently.metrics import RegressionTopErrorMetric
from evidently.metrics import RegressionErrorBiasTable
from evidently.metrics import RegressionDummyMetric
#from evidently.metrics import EmbeddingsDriftMetric

C:\Users\andre\miniconda3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<b>Prepare a Dataset</b>

In [2]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan

C:\Users\andre\miniconda3\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
#Dataset for binary label and probabilistic classifcation
bcancer_data = datasets.load_breast_cancer(as_frame='auto')
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

bcancer_label_ref = bcancer_ref.copy(deep=True)
bcancer_label_cur = bcancer_cur.copy(deep=True)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['prediction'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['prediction'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]

bcancer_label_ref['prediction'] = model.predict(bcancer_label_ref[bcancer_data.feature_names.tolist()])
bcancer_label_cur['prediction'] = model.predict(bcancer_label_cur[bcancer_data.feature_names.tolist()])

In [4]:
#Dataset for regression
housing_data = datasets.fetch_california_housing(as_frame='auto')
housing = housing_data.frame

housing.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing['prediction'] = housing_data['target'].values + np.random.normal(0, 3, housing.shape[0])

housing_ref = housing.sample(n=5000, replace=False)
housing_cur = housing.sample(n=5000, replace=False)

<b>How to run Reports?</b> <br>
<em> Data Drift Metrics</em>

In [5]:
#dataset-level metrics
data_drift_dataset_report = Report(metrics=[
    DataDriftTable(num_stattest='kl_div', cat_stattest='psi'),    
])

data_drift_dataset_report.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_dataset_report

In [7]:
#report in a JSON format
data_drift_dataset_report.json()
#report as a python object
data_drift_dataset_report.as_dict()

{'metrics': [{'metric': 'DataDriftTable',
   'result': {'number_of_columns': 15,
    'number_of_drifted_columns': 3,
    'share_of_drifted_columns': 0.2,
    'dataset_drift': False,
    'drift_by_columns': {'age': {'column_name': 'age',
      'column_type': 'num',
      'stattest_name': 'Kullback-Leibler divergence',
      'stattest_threshold': 0.1,
      'drift_score': 0.02723374709437682,
      'drift_detected': False,
      'current': {'small_distribution': {'x': [17.0,
         24.3,
         31.6,
         38.9,
         46.2,
         53.5,
         60.8,
         68.1,
         75.4,
         82.7,
         90.0],
        'y': [0.02471021672878118,
         0.025839691234843417,
         0.0262859521410848,
         0.025211766596857754,
         0.015942967066340047,
         0.010173168977679455,
         0.0061528716099474344,
         0.0018640278561586543,
         0.000568686464590777,
         0.0002369526935794904]}},
      'reference': {'small_distribution': {'x': [17.0

In [8]:
#column-level metrics
data_drift_column_report = Report(metrics=[
    ColumnDriftMetric('age'),
    ColumnValuePlot('age'),  
])

data_drift_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_column_report

<em>Data Integrity Metrics</em>

In [9]:
#dataset-level metrics
data_integrity_dataset_report = Report(metrics=[
    DatasetSummaryMetric(),
    DatasetMissingValuesMetric()
    
])

data_integrity_dataset_report.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_dataset_report

In [10]:
#column-level metrics
data_integrity_column_report = Report(metrics=[
    ColumnRegExpMetric(column_name="relationship", reg_exp=r".*child.*"),
    ColumnSummaryMetric(column_name="age"),
    ColumnMissingValuesMetric(column_name="education"),

    
])

data_integrity_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_column_report

<em>Classification Metrics</em>

In [11]:
#label binary classification
classification_report = Report(metrics=[
    ClassificationQualityMetric(),
    ClassificationClassBalance(),
    ConflictTargetMetric(),
    ConflictPredictionMetric(),
    ClassificationConfusionMatrix(),
    ClassificationQualityByClass(),
    ClassificationDummyMetric(),
    ClassificationQualityByFeatureTable(columns=['mean area', 'fractal dimension error']),
])

classification_report.run(reference_data=bcancer_label_ref, current_data=bcancer_label_cur)
classification_report

In [12]:
#probabilistic binary classification
classification_report = Report(metrics=[
    ClassificationQualityMetric(),
    ClassificationClassBalance(),
    ConflictTargetMetric(),
    ConflictPredictionMetric(),
    ClassificationConfusionMatrix(),
    ClassificationQualityByClass(),
    ClassificationClassSeparationPlot(),
    ClassificationProbDistribution(),
    ClassificationRocCurve(),
    ClassificationPRCurve(),
    ClassificationPRTable(),
    ClassificationQualityByFeatureTable(columns=['mean area', 'fractal dimension error']),

    
])

classification_report.run(reference_data=bcancer_ref, current_data=bcancer_cur)
classification_report

<em>Regreission Metrics</em>

In [14]:
regression_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionPredictedVsActualScatter(),
    RegressionPredictedVsActualPlot(),
    RegressionErrorPlot(),
    RegressionAbsPercentageErrorPlot(),
    RegressionErrorDistribution(),
    RegressionErrorNormality(),
    RegressionTopErrorMetric(),
    RegressionErrorBiasTable(columns=['MedInc', 'AveRooms']),
    RegressionDummyMetric(),
    ConflictTargetMetric(),
    ConflictPredictionMetric(),
    
])

regression_report.run(reference_data=housing_ref, current_data=housing_cur)
regression_report

<em>How to set metric parameters?</em>

In [15]:
#simple metric parameters
data_integrity_column_report = Report(metrics=[
    ColumnRegExpMetric(column_name="education", reg_exp=r".*-.*", top=5),
    ColumnRegExpMetric(column_name="relationship", reg_exp=r".*child.*")
])

data_integrity_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_integrity_column_report

In [16]:
#options
color_scheme = ColorOptions(
    primary_color="#5a86ad",
    fill_color="#fff4f2",
    zero_line_color="#016795",
    current_data_color= "#c292a1" ,
    reference_data_color="#017b92",
)

In [17]:
data_drift_column_report = Report(metrics=[
    ColumnDriftMetric('age'),
    ColumnDriftMetric('age', stattest='psi'),
],
                                  options=[color_scheme]
)

data_drift_column_report.run(reference_data=adult_ref, current_data=adult_cur)
data_drift_column_report